## Import

In [2]:
import pandas as pd
from statsmodels.stats import api as sms
import numpy as np
import math 

## Loading

In [3]:
df_raw=pd.read_csv('C:/Users/Utente77/repos/AB_Test/dataset/ab_data.csv')

In [4]:
df_raw.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [5]:
df_raw.shape

(294478, 5)

## DOE

### Hypothesys

### Parametres

In [6]:
## Nível de confiança:
confidence_level = 0.95
    
## Nível de significância:
significance_level= 0.05

## webpage conversion
p1=0.13
p2=0.15
p=0.1 ## conversao historica
                   
## Effect Size
effect_size=sms.proportion_effectsize(p1,p2)

## Poder Estatístico
statistic_power=0.8


## Sampling - Aleatoria simples

In [7]:
## sample size
sample_n=sms.NormalIndPower().solve_power( ## metade dos dados
    effect_size,
    power=statistic_power,
    alpha=significance_level
    )

sample_n=math.ceil(sample_n)

In [8]:
sample_n ## controle

4720

In [9]:
sample_total= 2*sample_n # samplig de p1 e p2
sample_total

9440

In [13]:
#sabendo que temos 10% de abertura mail para obter a amostra necessaria precisamos enviar 110% a mais
n_invio_mail=sample_n/p
n_invio_mail

47200.0